<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/Prepare_Labeled_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install


#init Repository - OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


In [ ]:
from numpy.core.fromnumeric import shape


import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


#Path wo das File mit den Keypoints + Label gespeichert wird
json_path = "/content.gdrive/MyDrive/00_2022-11-25_MCI/Dataset"

# dictionary to store mapping, labels, and Keypoints
dataset = {
    "labels":[],
    "keypoints":[]
}

#Path der Bilder - am besten Ordnerstruktur so:
  #Hauptordner 
      #Ordner mit Labels 
      #-FP 
          #Alle Bilder die dem zugeordnet sind
      #-LP 
          #Alle Bilder die dem zugeordnet sind
      #-LF
          #Alle Bilder die dem zugeordnet sind


PATH = "/content.gdrive/MyDrive/00_2022-11-25_MCI/Hauptordner"
Path_Result = "/content.gdrive/MyDrive/00_2022-11-25_MCI/Result"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#Ordner der Lables
print(label)

savecounter = 0

for l in label:
  Path_Pics = PATH + '/' + l
  print(Path_Pics)
  Pics = []
  try:
    Pics = next(os.walk(Path_Pics))[2]
  except StopIteration:
    pass
    #Ordner der Sprünge
  print("Anzahl: " , len(Pics))
  

  for p in Pics:
        Path_Pic = Path_Pics + '/' + p
        print(Path_Pic)

        size = len(p)
        p = p[:size - 4]

        %cd /content/yolov5
        !python detect.py --weights "/content/yolov5/weights_skijumper_only.pt" --name '{p}' --save-crop --source '{Path_Pic}' --project '{Path_Result}' --save-txt --max-det 1




        
        Path_crop = Path_Result + '/' + p + '/crops'
        %cd /content/openpose/build/examples/tutorial_api_python
        !python3 04_keypoints_from_images.py --image_dir '{Path_crop}' --keypoint_scale 3 --no_display true --write_images '{Path_crop}'  #keypoint_scale 0für nicht normalisiert

      
        # im Ordner Result/xxx/crops muss ein json File eingelesen und in das Dictionary mit Label eingefügt werden

        with open(Path_crop + '/keypoints.json', "r") as fp:
            keypoints_json = json.load(fp)
        dataset["keypoints"].append(keypoints_json)
        
        if l == 'FP':
            dataset["labels"].append(0)
        if l == 'LF':
            dataset["labels"].append(1) 
        if l == 'LP':
            dataset["labels"].append(2)
        
        """
        
        savecounter +=1

        if savecounter%1 == 0:
            #json_path_1 = json_path + 'with_BB_' + str(savecounter-99) + '_' + str(savecounter) + '.json'
            #with open(json_path_1, "w") as fp:
                #json.dump(dataset, fp, indent=4)

            json_path_2 = Path_Pic + '/' + str(savecounter) + '.json'
            with open(json_path_2, "w") as fp:
                json.dump(dataset2, fp, indent=4)

            # dictionary to store mapping, labels, and Keypoints leeren
            dataset = {
                "labels":[],
                "keypoints_with_BB":[]
            }

        """
json_path_2 = json_path + '/Dataset.json'
with open(json_path_2, "w") as fp:
  json.dump(dataset, fp, indent=4)  


['FP', 'LP', 'LF']
/content.gdrive/MyDrive/00_2022-11-25_MCI/Hauptordner/FP
Anzahl:  57
/content.gdrive/MyDrive/00_2022-11-25_MCI/Hauptordner/FP/Cam1_0146.jpg
/content/yolov5
detect: weights=['/content/yolov5/weights_skijumper_only.pt'], source=/content.gdrive/MyDrive/00_2022-11-25_MCI/Hauptordner/FP/Cam1_0146.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content.gdrive/MyDrive/00_2022-11-25_MCI/Result, name=Cam1_0146, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.1-578-ge4fa487 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content.gdrive/MyDrive/00_2022-11-25_MCI/Hauptordner/FP/Cam1